# Example code for search of best activation function
### [----->ML From Scratch](https://mlfromscratch.com)
We are going to run a small experiment, using mostly Keras and Scikit-Learn. The goal is to take the MNIST dataset and apply a search over the best hyperparameters - in particular, the best activation functions.

The outline for this notebook is as follows:
1. Importing packages and small tweaks to MNIST dataset
2. Readying functions, hyperparameters etc.
3. Running a GridSearchCV

In [24]:
# IMPORTS
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# LOAD DATA
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# PREPROCESSING
def preprocess_mnist(x_train, y_train, x_test, y_test):
    # Normalizing all images of 28x28 pixels
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)
    
    # Float values for division
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Normalizing the RGB codes by dividing it to the max RGB value
    x_train /= 255
    x_test /= 255
    
    # Categorical y values
    y_train = to_categorical(y_train)
    y_test= to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test, input_shape
    
x_train, y_train, x_test, y_test, input_shape = preprocess_mnist(x_train, y_train, x_test, y_test)

In [27]:
# Readying neural network model
def build_cnn(activation='relu',
              dropout_rate=0.1):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model

parameter_grid = {
                'epochs' :              [100,200],
                'batch_size' :          [32],
                'optimizer' :           ['Adam'],
                'dropout_rate' :        [0.1, 0.3, 0.5],
                'activation' :          ['relu', 'elu']
                }

GCV = GridSearchCV(estimator=KerasClassifier(build_fn=build_cnn,verbose=0),
                   param_grid=parameter_grid,
                   scoring='accuracy',
                   n_jobs=-1,
                   cv=5
                   )